### Initially, the comments were written in Russian. Translated by Google Translator

### Изначально комментарии были написаны на русском языке. Переведены через Google-переводчик

# Evaluating the effectiveness of a new feature
## Tasks:
 - 1 Calculate Retention Rate indicators of the seventh day before and after the implementation of the feature.
 - 2 Calculate ARPPU before and after the implementation of the feature.

# Оценка эффективности новой фичи
## Задачи:
 - 1 Посчитать показатели Retention Rate седьмого дня до и после внедрения фичи.
 - 2 Посчитать ARPPU до и после внедрения фичи.

In [1]:
# importing the libraries needed for the work
# импортируем нужные для работы библиотеки
import pandas as pd
import psycopg2
import psycopg2.extras

In [2]:
#  request data (registration date and number of unique users) on the number of registered users for each day of the studied period
# запрашиваем данные (дату регистрации и количество уникальных поьзователей) по количеству зарегистрировавшихся пользователей на каждый день исследуемого переода
def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count_df = pd.DataFrame(getRegisteredUsersCount())

# check the correctness of the received data
# проверяем корректность полученых данных
registered_users_count_df.head()

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787


In [3]:
# check the correctness of the received data
# проверяем корректность полученых данных
registered_users_count_df.head()

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787


In [4]:
# requesting data from the combined user registration and activity tables
# запрашиваем данные из обьеденённых таблиц регистрации и активности пользователей 
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data 
active_users_count_with_cohorts_df = pd.DataFrame(getActiveUsersCountWithCohorts())

# checking data
# проверяем данные
active_users_count_with_cohorts_df.head()

,activity_date,registration_date,active_users_count
0,2019-05-01,2019-05-01,2590
1,2019-05-02,2019-05-01,2146
2,2019-05-02,2019-05-02,2049
3,2019-05-03,2019-05-01,1637
4,2019-05-03,2019-05-02,1817


In [5]:
# combine the dateframes and check data
# объединяем датафреймы и проверяем данные
retention_table_df = active_users_count_with_cohorts_df.merge(registered_users_count_df,on=['registration_date'],how='left') 
retention_table_df.head()

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-05-01,2019-05-01,2590,6516
1,2019-05-02,2019-05-01,2146,6516
2,2019-05-02,2019-05-02,2049,5309
3,2019-05-03,2019-05-01,1637,6516
4,2019-05-03,2019-05-02,1817,5309


In [6]:
# convert the data in the combined table to the datetime type
# преобразуем данные в объеденнной таблице в тип datatime
for col in ['activity_date','registration_date']:
    retention_table_df[col] = pd.to_datetime(retention_table_df[col])

In [7]:
# calculate the retention rate in percentages
# посчитаем коэффициент удержания в проценах
retention_table_df['retention_rate(%)'] = retention_table_df['active_users_count'] / retention_table_df['registered_users_count']*100

# counting the day of the cohort's life
# считаем день жизни когорты
retention_table_df['lifetime'] = retention_table_df['activity_date'] - retention_table_df['registration_date']

retention_table_df

,activity_date,registration_date,active_users_count,registered_users_count,retention_rate(%),lifetime
0,2019-05-01,2019-05-01,2590,6516,39.748312,0 days
1,2019-05-02,2019-05-01,2146,6516,32.934316,1 days
2,2019-05-02,2019-05-02,2049,5309,38.594839,0 days
3,2019-05-03,2019-05-01,1637,6516,25.122775,2 days
4,2019-05-03,2019-05-02,1817,5309,34.224901,1 days
...,...,...,...,...,...,...
1886,2019-07-28,2019-06-29,129,2968,4.346361,29 days
1887,2019-07-28,2019-06-30,198,3301,5.998182,28 days
1888,2019-07-29,2019-06-29,89,2968,2.998652,30 days
1889,2019-07-29,2019-06-30,144,3301,4.362314,29 days


In [8]:
# request data on purchases: date, cohort by registration date, number of paying users, amount of revenue
# запрашиваем данные по покупкам: дата, когорта по дате регистрации, количество платящих пользователей, сумма выручки
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
revenue_df = pd.DataFrame(getRevenue())

revenue_df.head()

,date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.590
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608


In [9]:
# converting dates to datetime type
# приводим даты к типу datetime

for col in ['date','registration_date']:
    revenue_df[col] = pd.to_datetime(revenue_df[col])

In [10]:
# for convenience, rename the date column
# для удобства переименовываем столбец date
revenue_df = revenue_df.rename(columns={'date':'activity_date'})

# combining dateframes
# объединяем датафреймы 
retention_table_with_revenue_df = retention_table_df.merge(revenue_df,on=['registration_date','activity_date'],how='left')

In [11]:
# we process the omissions by replacing them with zero
# обрабатываем пропуски заменяя их на ноль

for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue_df[col] = retention_table_with_revenue_df[col].fillna(0)

In [12]:
# calculate the average revenue from the paying user
# считаем средную выручку с платящего пользователя 
retention_table_with_revenue_df['arppu'] = (retention_table_with_revenue_df['revenue'] / retention_table_with_revenue_df['users_count_with_revenue']) * 100

In [13]:
# select all indicators of the seventh day from the retention_table_with_revenue dataframe
# выделяем из датафрейма retention_table_with_revenue все показатили седьмого дня 
retention_table_7days_df = retention_table_with_revenue_df[retention_table_with_revenue_df['lifetime'] == '7 days']

# spliting it into two tables, with indicators for May and June
# разделяем на две таблицы, с показателями за май и июнь
retention_table_7days_may_df = retention_table_7days_df[retention_table_7days_df['registration_date'] < '2019-06-01']
retention_table_7days_june_df = retention_table_7days_df[retention_table_7days_df['registration_date'] >= '2019-06-01']

In [14]:
print('Средний показатель коэффициента удержания седьмого день составляют:',
      '\n май', round(retention_table_7days_may_df['retention_rate(%)'].mean(), 2),'%',
      '\n июнь', round(retention_table_7days_june_df['retention_rate(%)'].mean(), 2),'%',
      '\n их соотношение:',round(retention_table_7days_june_df['retention_rate(%)'].mean()/retention_table_7days_may_df['retention_rate(%)'].mean(), 2))

print('Средний месячный доход с активного пользователясоставляет:',
      ' \n май', round(retention_table_with_revenue_df.query('activity_date < "2019-06-01"').arppu.mean(), 2),'%',
      '\n июнь', round(retention_table_with_revenue_df.query('activity_date >= "2019-06-01"').arppu.mean(), 2),'%',
      '\n их соотношение:', round(retention_table_with_revenue_df.query('activity_date >= "2019-06-01"').arppu.mean()/retention_table_with_revenue_df.query('activity_date < "2019-06-01"').arppu.mean(), 2))

Средний показатель коэффициента удержания седьмого день составляют: 
 май 12.83 % 
 июнь 12.52 % 
 их соотношение: 0.98
Средний месячный доход с активного пользователясоставляет:  
 май 25.99 % 
 июнь 28.55 % 
 их соотношение: 1.1


# Сonclusion:

1 The retention coefficient has not actually changed in the month after the introduction of the feature. Most likely, users are not yet used to the new functionality, it may be worth analyzing for a longer period of time.

2 With the introduction of new functionality, ARPU grew by 2.5%, which is a good indicator. In order to understand whether the introduction of the feature and the increase in ARPU are connected, it is necessary to conduct a study on a longer time interval.

# Вывод:

1 Коффициент удержания фактически не изменился за месяц после введения фичи. Скорее всего пользователи ещё не привыкли к новому функционалу, возможно, стоит провести анализ на большем промежутке времени.

2 C внедрением нового функционала ARPU вырос на 2.5%, что является хорошим показателем. Что бы понять связано ли введение фичи и увеличение ARPU, нужно провести исследование на большем временном отрезке.